In [13]:
!pip3 install requests



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

print("--- Solving Question 1: Scraping 4 Book Categories ---")

# Base URL for the website
base_url = "https://books.toscrape.com/"
all_books_data = []

# The specific category pages to scrape
categories = [
    'catalogue/category/books/travel_2/index.html',
    'catalogue/category/books/mystery_3/index.html',
    'catalogue/category/books/historical-fiction_4/index.html',
    'catalogue/category/books/sequential-art_5/index.html'
]

for category_path in categories:
    url = base_url + category_path
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    category_name = soup.find('h1').text
    print(f"Scraping category: {category_name}...")
    
    # Find all the book containers on the page
    books = soup.find_all('article', class_='product_pod')
    
    for book in books:
        title = book.find('h3').find('a')['title']
        price = book.find('p', class_='price_color').text
        stock = book.find('p', class_='instock availability').text.strip()
        # The star rating is in the class name, e.g., "star-rating Three"
        rating = book.find('p', class_='star-rating')['class'][1]

        all_books_data.append([title, price, stock, rating, category_name])

# Create a DataFrame and save to CSV
df = pd.DataFrame(all_books_data, columns=['Title', 'Price', 'Stock', 'Rating', 'Category'])
df.to_csv('books_by_category.csv', index=False)

print("\nScraping complete. Data saved to 'books_by_category.csv'.")

    

--- Solving Question 1: Scraping 4 Book Categories ---
Scraping category: Travel...
Scraping category: Mystery...
Scraping category: Historical Fiction...
Scraping category: Sequential Art...

Scraping complete. Data saved to 'books_by_category.csv'.


In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

print("--- Solving Question 2: Scraping IMDb's Top 250 Movies ---")

url = "https://www.imdb.com/chart/top/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

movies_data = []

movie_list_items = soup.select(".ipc-metadata-list-summary-item")

# Loop through ALL the items in the list we found (no [:50] slice).
for item in movie_list_items:
    # Find the <h3> tag which contains the movie's rank and title.
    title_tag = item.find("h3", class_="ipc-title__text")
    # The text looks like "1. The Shawshank Redemption". We split it at the dot and take the second part.
    name = title_tag.text.split(".")[1].strip()

    # The metadata div contains the year, duration, and rating.
    metadata_div = item.find("div", class_="cli-title-metadata")
    metadata_items = metadata_div.find_all("span", class_="cli-title-metadata-item")
    # The year is in the first span.
    year = metadata_items[0].text.strip()

    # The rating is in a specific div and span structure.
    rating_span = item.find("span", class_="ipc-rating-star")
    # The text inside looks like "9.3 (3.0M)". We split by space and take the first part.
    rating = rating_span.text.split(" ")[0]

    movies_data.append([name, year, rating])

df = pd.DataFrame(movies_data, columns=["Name", "Year", "Rating"])
df.to_csv("imdb_top250_movies.csv", index=False)

print(f"\nScraping complete. Found {len(movies_data)} movies.")
print("Data saved to 'imdb_top250_movies.csv'.")

--- Solving Question 2: Scraping IMDb's Top 250 Movies ---

Scraping complete. Found 25 movies.
Data saved to 'imdb_top250_movies.csv'.


In [20]:
!pip3 install selenium

  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 3.3 MB/s eta 0:00:00a 0:00:01
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.13.2
    Uninstalling typing_extensions-4.13.2:
      Successfully uninstalled typing_extensions-4.13.2
  Attempting uninstall: certifi
    Found existing installation: certifi 2024.12.14
    Uninstalling certifi-2024.12.14:
      Successfully uninstalled certifi-2024.12.14

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [25]:
!pip3 install webdriver-manager


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip


In [27]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

print("--- Solving Question 2 (Corrected): Scraping All 250 IMDb Movies ---")

# --- Part 1: Control the Browser with Selenium ---

# Setup and launch a Chrome browser automatically
# This will open a new Chrome window
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Go to the IMDb Top 250 URL
url = "https://www.imdb.com/chart/top/"
driver.get(url)
print("Browser opened. Waiting for page to load and scrolling...")

# Scroll down the page multiple times to load all movies
# We give it a few seconds to make sure all content loads
scroll_pause_time = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
    time.sleep(scroll_pause_time)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Get the complete HTML source code after everything has loaded
page_source = driver.page_source

# Close the browser window
driver.quit()
print("Scrolling complete and browser closed.")

# --- Part 2: Parse the Complete HTML with BeautifulSoup ---

# Turn the complete HTML into a searchable BeautifulSoup object
soup = BeautifulSoup(page_source, "html.parser")

movies_data = []
movie_list_items = soup.select(".ipc-metadata-list-summary-item")

# Loop through ALL the items in the list we found
for item in movie_list_items:
    title_tag = item.find("h3", class_="ipc-title__text")
    name = title_tag.text.split(".")[1].strip()

    metadata_div = item.find("div", class_="cli-title-metadata")
    metadata_items = metadata_div.find_all("span", class_="cli-title-metadata-item")
    year = metadata_items[0].text.strip()

    rating_span = item.find("span", class_="ipc-rating-star")
    rating = rating_span.text.split(" ")[0]

    movies_data.append([name, year, rating])

df = pd.DataFrame(movies_data, columns=["Name", "Year", "Rating"])
df.to_csv("imdb_top250_movies.csv", index=False)

print(f"\nScraping complete. Found {len(movies_data)} movies.")
print("Data saved to 'imdb_top250_movies.csv'.")

--- Solving Question 2 (Corrected): Scraping All 250 IMDb Movies ---
Browser opened. Waiting for page to load and scrolling...
Scrolling complete and browser closed.

Scraping complete. Found 250 movies.
Data saved to 'imdb_top250_movies.csv'.


In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

print("--- Solving Question 3 (Final Corrected Version): Scraping Weather Data ---")

url = "https://www.timeanddate.com/weather/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

weather_data = []

weather_table = soup.find("table", class_="zebra fw tb-theme")
city_rows = weather_table.find_all("tr")

# Loop through the first 11 rows, skipping the header with [1:11]
for row in city_rows[1:11]:
    cells = row.find_all("td")
    
    # Extract data that is always in the same place
    city = cells[0].text.strip()
    temperature = cells[2].text.strip()
    wind = cells[5].text.strip()
    
    # --- NEW LOGIC to handle the weather description ---
    # Get the cell where the weather description is located.
    weather_cell = cells[1]
    # First, look for an <img> tag inside this cell.
    img_tag = weather_cell.find('img')
    
    # IF an image tag exists, get the weather from its 'title'.
    if img_tag:
        weather = img_tag['title']
    # ELSE (if no image tag exists), get the weather from the cell's plain text.
    else:
        weather = weather_cell.text.strip()

    # Add the correctly found data to our list.
    weather_data.append([city, weather, temperature, wind])

df = pd.DataFrame(weather_data, columns=["City", "Weather", "Temperature", "Wind Speed"])
df.to_csv("world_weather.csv", index=False)

print("\nScraping complete. Data saved to 'world_weather.csv'.")




--- Solving Question 3 (Final Corrected Version): Scraping Weather Data ---

Scraping complete. Data saved to 'world_weather.csv'.
